# Galaxy Morphology Classification with Deep Learning

This notebook demonstrates galaxy morphology classification using convolutional neural networks (CNNs) and transfer learning on the Galaxy Zoo 2 dataset.

## The Science

**Galaxy morphology** - the shape and structure of galaxies - encodes crucial information about:
- Formation history (mergers vs. quiet accretion)
- Stellar populations (young vs. old stars)
- Environment (field vs. cluster galaxies)
- Dark matter distribution

### Hubble's Tuning Fork

Edwin Hubble (1926) created the first morphological classification scheme:

```
                    Sa ─── Sb ─── Sc ─── Sd (Spirals with bars: SBa, SBb...)
                   /
E0 ─ E3 ─ E7 ─ S0 ─
                   \
                    Irr (Irregular)
```

Today, we classify galaxies into:
- **Ellipticals (E)**: Smooth, featureless, red (old stars)
- **Spirals (S/SB)**: Disk + spiral arms, blue arms (star formation)
- **Lenticulars (S0)**: Disk but no arms (transition type)
- **Irregulars (Irr)**: No regular structure

---

In [ ]:
# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.insert(0, str(Path('.').resolve().parent))

# Deep learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Our modules
from src.preprocessing import GalaxyPreprocessor
from src.models import GalaxyCNN, create_transfer_model, GalaxyAutoencoder
from src.visualization import plot_galaxy_grid, plot_confusion_matrix, plot_training_history

# Settings
plt.style.use('seaborn-v0_8-whitegrid')
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

## 1. Data Loading and Exploration

We use synthetic data for this demo. The Galaxy Zoo 2 dataset can be downloaded using:
```bash
python scripts/download_data.py --dataset galaxy
```

In [ ]:
# Generate synthetic galaxy data for demo
np.random.seed(42)

def generate_synthetic_galaxy(galaxy_type, size=128):
    """Generate a synthetic galaxy image."""
    img = np.zeros((size, size, 3), dtype=np.float32)
    y, x = np.ogrid[-size//2:size//2, -size//2:size//2]
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)
    
    if galaxy_type == 0:  # Elliptical
        a = np.random.uniform(20, 40)
        b = np.random.uniform(10, a)
        angle = np.random.uniform(0, np.pi)
        x_rot = x * np.cos(angle) + y * np.sin(angle)
        y_rot = -x * np.sin(angle) + y * np.cos(angle)
        r_ell = np.sqrt((x_rot/a)**2 + (y_rot/b)**2)
        intensity = np.exp(-7.67 * (r_ell**(1/4) - 1))
        img[:,:,0] = intensity
        img[:,:,1] = intensity * 0.8
        img[:,:,2] = intensity * 0.6
        
    elif galaxy_type == 1:  # Spiral
        disk = 0.8 * np.exp(-r/30)
        n_arms = np.random.choice([2, 4])
        arms = 0.3 * np.sin(n_arms * theta - 0.3 * r) * np.exp(-r/40)
        arms = np.clip(arms, 0, 0.3)
        bulge = np.exp(-r**2/100)
        intensity = np.clip(disk + arms + bulge, 0, 1)
        img[:,:,0] = intensity * (0.7 + 0.3*np.exp(-r/20))
        img[:,:,1] = intensity
        img[:,:,2] = intensity * (0.5 + 0.5*(1-np.exp(-r/30)))
        
    elif galaxy_type == 2:  # Edge-on
        a = np.random.uniform(40, 55)
        b = np.random.uniform(3, 8)
        r_ell = np.sqrt((x/a)**2 + (y/b)**2)
        disk = 0.8 * np.exp(-r_ell)
        dust = 1 - 0.7 * np.exp(-y**2/4) * (np.abs(x) < 35)
        intensity = disk * dust
        img[:,:,0] = intensity
        img[:,:,1] = intensity * 0.9
        img[:,:,2] = intensity * 0.7
        
    else:  # Irregular
        n_clumps = np.random.randint(3, 7)
        for _ in range(n_clumps):
            cx = np.random.randint(size//4, 3*size//4)
            cy = np.random.randint(size//4, 3*size//4)
            sigma = np.random.uniform(5, 15)
            brightness = np.random.uniform(0.3, 0.8)
            clump = brightness * np.exp(-((x-cx+size//2)**2 + (y-cy+size//2)**2)/(2*sigma**2))
            color_idx = np.random.choice([0, 1, 2], p=[0.2, 0.3, 0.5])
            img[:,:,color_idx] = np.clip(img[:,:,color_idx] + clump, 0, 1)
    
    # Add noise
    noise = np.random.randn(size, size, 3) * 0.05
    img = np.clip(img + noise, 0, 1)
    
    return img.astype(np.float32)

# Generate dataset
n_per_class = 200
class_names = ['Elliptical', 'Spiral', 'Edge-on', 'Irregular']

images = []
labels = []

for class_id in range(4):
    for _ in range(n_per_class):
        img = generate_synthetic_galaxy(class_id)
        images.append(img)
        labels.append(class_id)

X = np.array(images)
y = np.array(labels)

print(f"Dataset shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Class distribution: {np.bincount(y)}")

In [ ]:
# Visualize examples from each class
fig, axes = plt.subplots(4, 5, figsize=(15, 12))

for i, class_name in enumerate(class_names):
    class_images = X[y == i][:5]
    for j, img in enumerate(class_images):
        axes[i, j].imshow(np.clip(img, 0, 1))
        axes[i, j].axis('off')
        if j == 0:
            axes[i, j].set_ylabel(class_name, fontsize=12, rotation=0, ha='right')

plt.suptitle('Galaxy Morphology Examples', fontsize=14)
plt.tight_layout()
plt.savefig('../images/galaxy_examples.png', dpi=150, bbox_inches='tight')
plt.show()

## 2. Data Preprocessing

Galaxy images require specific preprocessing:

1. **Background subtraction**: Remove sky brightness
2. **Centering**: Align galaxy to image center
3. **Logarithmic scaling**: Compress dynamic range (bright core, faint outskirts)
4. **Normalization**: Scale to [0, 1] range

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Transpose for PyTorch (N, H, W, C) -> (N, C, H, W)
X_train_torch = torch.FloatTensor(X_train.transpose(0, 3, 1, 2))
X_test_torch = torch.FloatTensor(X_test.transpose(0, 3, 1, 2))
y_train_torch = torch.LongTensor(y_train)
y_test_torch = torch.LongTensor(y_test)

# Create data loaders
train_dataset = TensorDataset(X_train_torch, y_train_torch)
test_dataset = TensorDataset(X_test_torch, y_test_torch)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## 3. Model Architecture

### Why CNNs for Galaxy Classification?

**Convolutional Neural Networks** are ideal for this task because:

1. **Local feature detection**: Convolutional filters detect local patterns
   - Edges and gradients → spiral arm structure
   - Circular patterns → bulge detection
   - Texture → star formation regions

2. **Translation equivariance**: Galaxies can appear anywhere in the image

3. **Hierarchical learning**: 
   - Early layers: Low-level features (edges, colors)
   - Middle layers: Textures, small structures
   - Late layers: Spiral arms, bars, bulges
   - Final layers: Overall morphology

### Why NOT U-Net or Segmentation Models?

- **U-Net** is designed for pixel-wise segmentation (e.g., separating galaxy from background)
- Our task is **classification**, not segmentation
- We need a single class label, not a pixel map
- Segmentation would be useful for: galaxy deblending, photometry, morphological measurements

In [ ]:
# Create custom CNN model
model = GalaxyCNN(n_classes=4, input_size=(128, 128), dropout=0.5)
model = model.to(DEVICE)

print("Model Architecture:")
print(model)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

## 4. Training

We use:
- **Cross-entropy loss**: Standard for multi-class classification
- **Adam optimizer**: Adaptive learning rate
- **Early stopping**: Prevent overfitting

In [ ]:
# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)

# Training loop
n_epochs = 30
history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

best_val_acc = 0
patience_counter = 0
patience = 10

for epoch in range(n_epochs):
    # Training
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0
    
    for batch_x, batch_y in train_loader:
        batch_x = batch_x.to(DEVICE)
        batch_y = batch_y.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_total += batch_y.size(0)
        train_correct += (predicted == batch_y).sum().item()
    
    # Validation
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x = batch_x.to(DEVICE)
            batch_y = batch_y.to(DEVICE)
            
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += batch_y.size(0)
            val_correct += (predicted == batch_y).sum().item()
    
    # Record metrics
    train_loss /= len(train_loader)
    val_loss /= len(test_loader)
    train_acc = train_correct / train_total
    val_acc = val_correct / val_total
    
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['train_acc'].append(train_acc)
    history['val_acc'].append(val_acc)
    
    scheduler.step(val_loss)
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}: "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    
    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        # Save best model
        torch.save(model.state_dict(), '../models/galaxy_cnn_best.pt')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

print(f"\nBest validation accuracy: {best_val_acc:.4f}")

In [ ]:
# Plot training history
fig = plot_training_history(history)
plt.savefig('../images/galaxy_training_history.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Evaluation

### Interpretation of Results

The confusion matrix shows:
- **Diagonal elements**: Correct classifications
- **Off-diagonal elements**: Misclassifications

Common confusions in galaxy classification:
- Elliptical ↔ Edge-on: Viewed at different angles, similar profiles
- Spiral ↔ Irregular: Disturbed spirals can look irregular
- Face-on S0 ↔ Elliptical: S0 galaxies without obvious disk features

In [ ]:
# Load best model and evaluate
model.load_state_dict(torch.load('../models/galaxy_cnn_best.pt'))
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x = batch_x.to(DEVICE)
        outputs = model(batch_x)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(batch_y.numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Classification report
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))

In [ ]:
# Confusion matrix
fig = plot_confusion_matrix(all_labels, all_preds, class_names=class_names)
plt.savefig('../images/galaxy_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Transfer Learning Comparison

Transfer learning often outperforms training from scratch because:
- Pre-trained networks have robust low-level features
- Requires less training data
- Faster convergence

In [ ]:
# Compare with transfer learning (if torchvision is available)
try:
    transfer_model = create_transfer_model(n_classes=4, backbone='resnet18', pretrained=True)
    transfer_model = transfer_model.to(DEVICE)
    print("Transfer learning model created successfully")
    print(f"Parameters: {sum(p.numel() for p in transfer_model.parameters()):,}")
    print(f"Trainable: {sum(p.numel() for p in transfer_model.parameters() if p.requires_grad):,}")
except Exception as e:
    print(f"Transfer learning model not available: {e}")

## 7. Conclusions

### Key Findings

1. **CNNs effectively classify galaxy morphology** from imaging data
2. **Transfer learning** provides robust features even from non-astronomical pre-training
3. **Common confusions** occur between morphologically similar classes
4. **Data augmentation** (rotation, flips) is physically valid for galaxies

### Limitations

- **Synthetic data**: Real galaxies have more diverse morphologies
- **Simplified classes**: Galaxy Zoo uses continuous vote fractions, not discrete classes
- **Redshift effects**: Distant galaxies appear smaller and fainter

### Future Directions

- Use real Galaxy Zoo 2 data with vote fractions as soft labels
- Explore attention mechanisms for interpretability
- Multi-task learning: morphology + redshift estimation